first we have to install the library [rdflib](https://rdflib.readthedocs.io/en/stable/) and import it next to [pandas](https://pandas.pydata.org/), which helbs to work with a csv file. 

In [1]:
!pip install rdflib
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import DCTERMS, RDF, RDFS, SKOS, XSD
import pandas as pd

     |████████████████████████████████| 235kB 6.6MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 


`result_graph` contains the rdf graph we want to generate and serialize in a new output file. we predefine alle needed namespaces and add them to `result_graph`.

In [2]:
#
result_graph = Graph()

GAMS = Namespace("https://gams.uni-graz.at/o:gams-ontology#") 
VOID = Namespace("http://rdfs.org/ns/void#")
FOAF = Namespace("http://xmlns.com/foaf/spec/")
DCTERMS = Namespace("http://purl.org/dc/terms/")
DC = Namespace("http://purl.org/dc/elements/1.1/")
OL = Namespace("http://olympia-ontology#")

result_graph.bind("gams", GAMS)
result_graph.bind("void", VOID)
result_graph.bind("foaf", FOAF)
result_graph.bind("dcterms", DCTERMS)
result_graph.bind("dc", DC)
result_graph.bind("ol", OL)

# so that https://gams.uni-graz.at/olympia.1 is the URL of our RDF document somewhere in the web
BASE_URL = "https://gams.uni-graz.at/"
DOCUMENT_ID = "olympia.1"

loading the data into a new panda dataframe. upload the `athlete_events_smaller.csv` into the *data* folder. the csv contains 2500 athletes for each multiple rows can exist. the header of the CSV looks like the following;

"ID","Name","Sex","Age","Height","Weight","Team","NOC","Games","Year","Season","City","Sport","Event","Medal"

**YOU HAVE TO REUPLOAD THE CSV INPUT INTO THE CORRECT FOLDER "content" !!!**

In [4]:
df = pd.read_csv("athlete_events_smaller.csv", encoding="utf8") 
print(df)

FileNotFoundError: ignored

groups all rows by "ID" and add properties to ol:Athlete

In [ ]:
#for group in df.groupby('ID'):
#    print(group)

groups_by_id = df.groupby('ID')
#print(BASE_URL)

for ID, group in groups_by_id:
  # create <ol:Athlete rdf:about="https://gams.uni-graz.at/olympia.1#9792"/>
  athlete = URIRef(BASE_URL + DOCUMENT_ID + "#" + str(ID))
  result_graph.add((athlete, RDF.type, OL.Athlete))
  
  # second loop for every group
  for datafield in group:
    if(datafield == "Name"):
      result_graph.add(( athlete, FOAF.name, Literal(group[datafield].values[0]) ))
    if(datafield == "Height"):
      result_graph.add(( athlete, OL.height, Literal(group[datafield].values[0]) ))
    if(datafield == "Weight"):
      result_graph.add(( athlete, OL.weight, Literal(group[datafield].values[0]) ))
    if(datafield == "Age"):
      result_graph.add(( athlete, OL.age, Literal(group[datafield].values[0]) ))
    if(datafield == "Sex"):
      result_graph.add(( athlete, FOAF.gender, Literal(group[datafield].values[0]) ))
    if(datafield == "Team"):
      result_graph.add(( athlete, OL.team, Literal(group[datafield].values[0]) ))

    #  
    if(datafield == "Season"):
      # ol:Result rdf:about="https://gams.uni-graz.at/olympia.1#144Result" , ol:season, ol:year, ol:discipline
      result = URIRef(BASE_URL + DOCUMENT_ID + "#" + str(ID) + "Result")
      result_graph.add((result, RDF.type, OL.Result))
      result_graph.add(( athlete, OL.result, result ))

      result_graph.add(( result, OL.season, Literal(group["Season"].values[0]) ))
      result_graph.add(( result, OL.year, Literal(group["Year"].values[0]) ))

      # <ol:Discipline rdf:about="https://gams.uni-graz.at/olympia.1#144Discipline">
      discipline = URIRef(BASE_URL + DOCUMENT_ID + "#" + str(ID) + "Discipline")
      result_graph.add(( result, OL.discipline, discipline ))
      result_graph.add(( discipline, RDF.type, OL.Discipline ))

      result_graph.add(( discipline, OL.sport,  Literal(group["Sport"].values[0])  ))
 

create new file
* format="xml" - creates plain xml/rdf
* format="pretty-xml" - abbreviated RDF/XML syntax 
+ format="turtle"

In [ ]:
result_graph.serialize(destination = 'output.xml', format="pretty-xml")